In [1]:
import pandas as pd
from model_tools import Preprocess
import os
from tqdm.notebook import tqdm

In [2]:
# convert 18K to 10K axis
axis_10K = pd.read_csv('P:\\DATASETS\\10K\\final_datasets\\10K_spectra_EARTH_9mJ.csv', usecols=[0])

folder = 'P:\\DATASETS\\18K\\final_datasets'
outfolder = 'P:\\DATASETS\\18K\\final_datasets\\10K axis_AirPLS_norm3\\'
file_list = [f for f in os.listdir(folder) if 'spectra' in f]
for file in tqdm(file_list):
    spectra = pd.read_csv(f'{folder}\\{file}')
    
    spectra_resampled = Preprocess.resample_to_match(spectra, axis_10K)
    
    # BLR
    spectra_blr = Preprocess.AirPLS(spectra_resampled)

    # norm5
    spectra_norm = Preprocess.norm3_SL_10K(spectra_blr)
    spectra_norm.to_csv(f'{outfolder}\\{file[:-4]}_formatted.csv', index=False)

  0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
inst_list = ['10K','18K']

for inst in inst_list:
    folder = f'P:\\DATASETS\\{inst}\\final_datasets'
    outfolder = f'{folder}\\SC axis_AirPLS_norm5'

    spec_files = [f for f in os.listdir(folder) if 'spectra' in f]

    for file in tqdm(spec_files):
        df = pd.read_csv(f'{folder}\\{file}')

        # resample to SC
        df_resampled = Preprocess.resample_to_match(df)

        # not going to resample to min step first
        # BLR
        df_blr = Preprocess.AirPLS(df_resampled)

        # norm5
        df_norm = Preprocess.norm5_SC(df_blr)

        df_norm.to_csv(f'{outfolder}\\{file[:-4]}_formatted.csv', index=False)

  0%|          | 0/16 [00:00<?, ?it/s]

Resampling to SuperCam -2px shifted wavelength axis by default
Resampling to SuperCam -2px shifted wavelength axis by default
Resampling to SuperCam -2px shifted wavelength axis by default
Resampling to SuperCam -2px shifted wavelength axis by default
Resampling to SuperCam -2px shifted wavelength axis by default
Resampling to SuperCam -2px shifted wavelength axis by default
Resampling to SuperCam -2px shifted wavelength axis by default
Resampling to SuperCam -2px shifted wavelength axis by default
Resampling to SuperCam -2px shifted wavelength axis by default
Resampling to SuperCam -2px shifted wavelength axis by default
Resampling to SuperCam -2px shifted wavelength axis by default
Resampling to SuperCam -2px shifted wavelength axis by default
Resampling to SuperCam -2px shifted wavelength axis by default
Resampling to SuperCam -2px shifted wavelength axis by default
Resampling to SuperCam -2px shifted wavelength axis by default
Resampling to SuperCam -2px shifted wavelength axis by 

  0%|          | 0/4 [00:00<?, ?it/s]

Resampling to SuperCam -2px shifted wavelength axis by default
Resampling to SuperCam -2px shifted wavelength axis by default
Resampling to SuperCam -2px shifted wavelength axis by default
Resampling to SuperCam -2px shifted wavelength axis by default
